In [2]:
import pandas as pd
import numpy as np

In [4]:
master = pd.read_csv("/Users/premsundar/Downloads/DA/cyclist/master.csv")

In [5]:
master.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0,D90034EFC7235A41,electric_bike,2025-12-22 17:20:27.872,2025-12-22 17:27:29.937,Broadway & Berwyn Ave,CHI00414,Broadway & Sunnyside Ave,CHI02089,41.978361,-87.659789,41.963419,-87.656069,member
1,1,B005230EE77A29D0,classic_bike,2025-12-20 17:15:31.749,2025-12-20 17:20:07.746,Sheridan Rd & Montrose Ave,CHI00290,Broadway & Sheridan Rd,CHI00476,41.961670,-87.654640,41.952833,-87.649993,member
2,2,FD33124F2231E215,classic_bike,2025-12-06 13:38:18.275,2025-12-06 13:52:54.682,Southport Ave & Wrightwood Ave,CHI00339,Larrabee St & Menomonee St,CHI00262,41.928773,-87.663913,41.914680,-87.643320,casual
3,3,7363165DB4BD6D14,electric_bike,2025-12-16 20:58:28.703,2025-12-16 21:04:35.476,Sheridan Rd & Montrose Ave,CHI00290,Clark St & Newport St,CHI00674,41.961670,-87.654640,41.944540,-87.654678,member
4,4,F09F745E43DF9556,electric_bike,2025-12-09 14:53:43.347,2025-12-09 14:58:30.944,Larrabee St & Kingsbury St 1,CHI00245,Rush St & Superior St,CHI00368,41.897764,-87.642884,41.895765,-87.625908,member


In [7]:
master.dtypes

Unnamed: 0              int64
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [9]:
master["started_at"] = pd.to_datetime(master["started_at"])
master["ended_at"] = pd.to_datetime(master["ended_at"])

In [10]:
master.dtypes

Unnamed: 0                     int64
ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

In [11]:
master.shape

(5552994, 14)